# Test Simple Maze

In [1]:
from environments import SimpleMaze

In [2]:
HIGHER_DIM_OBS = False
env = SimpleMaze(higher_dim_obs=HIGHER_DIM_OBS)

In [3]:
state = env.reset()

In [4]:
import numpy
numpy.set_printoptions(threshold=float('inf'))
print(state.shape)

(8, 8)


In [5]:
env.action_space.n

4

In [6]:
state

array([[1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ],
       [1. , 0. , 0. , 0. , 0.5, 0. , 0. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ]])

In [25]:
next_state, reward, done, _ = env.step(2)

In [26]:
next_state

array([[1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ],
       [1. , 0. , 0.5, 0. , 0. , 0. , 0. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ],
       [1. , 0. , 0. , 0. , 1. , 0. , 0. , 1. ],
       [1. , 1. , 1. , 1. , 1. , 1. , 1. , 1. ]])